# <font color="navy"> Lecture 5: Logistic Regression


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

data_path = "../data/"
Default = pd.read_csv(data_path + "Default.csv")

## 5.1 분류분석(Classification) 이란?
- 분류되어 있는 집단 간의 차이를 의미있게 설명해 줄 수 있는 독립변수들을 찾음
- 예
    - 서비스 이용 불만 고객의 성향 분석
    - SKT/KT/LGT 가입고객 판별 변수 및 판별함수 유도

- 비교
    - 군집분석
        - 관측치를 그룹으로 구분함
        - 분류분석과는 다르게 데이터에 집단을 나타내는 변수 없음
    - 회귀분석
        - 종속변수가 이산형인 logistic regression은 두 집단 분류분석의 한 방법
        - 분류분석은 집단이 2개 이상의 범주를 갖는 범주형변수인 경우 가능
        - 회귀분석은 종속변수가 연속형일 경우 사용 

#### Example: Default data
- 개인의 연봉과 월 신용카드 잔고를 사용해 파산 예측
    
<img src='http://drive.google.com/uc?export=view&id=1P9a2_d_iVhgr4b6bWhvj-IOp_I1VMKKg' width=40% align="left"/> <img src='http://drive.google.com/uc?export=view&id=1yci-b04K3LqKGEGyId_hKEL-HS4A7thm' width=40% align="left"/>

- 선형회귀모형은 적당치 않음

<img src='http://drive.google.com/uc?export=view&id=1KEcn9veVn0QLXHCIo9CZyU91YRZoOPjE' /><br>

## 5.2 Simple Logistic Regression
- Balance를 사용해 default=Yes일 확률 예측
    $$ p(X)=\frac{e^{\beta_0 + \beta_1 X}}{1+e^{\beta_0 + \beta_1 X}}    $$
    $$ \log \left( \frac{p(X)}{1-p(X)}\right) = \beta_0 + \beta_1 X $$
    - $\frac{p(X)}{1-p(X)}$: Odds
    - $\log \left( \frac{p(X)}{1-p(X)}\right) = logit(p(X))$

#### Odds란?
- 앞면이 나올 확률이 p인 동전을 던져 앞면이 나오면 $X를 받고, 뒷면을 나오면 $Y를 잃는 게임을 한다고 하자. 
이 게임이 공정하려면 X와 Y의 관계는?
$$E(\mbox{earning})=Xp-Y(1-p)=0$$
$$\frac Y X = \frac p {1-p} = Odds$$
- Odds의 의미
    - "p의 확률로 \$1을 받을 수 있다면(X=1) 1-p의 확률로 얼만큼을 잃을 의향이 있는가?"
    - 만일 앞면이 나올 확률이 0.9라면(p=0.9) 뒷면이 나올 때 \$9(=0.9/0.1)까지는 잃어도 ok
    - 만일 앞면이 나올 확률이 0.1이라면(p=0.1) 뒷면이 나올 때 $1/9(=0.1/0.9)까지는 잃어도 ok

#### 회귀계수의 해석     

- $\beta_1>0$: X가 증가하면 $p(X)$가 증가
- $\beta_1<0$: X가 증가하면 $p(X)$가 감소
- $\beta_0$: Balance가 0일 때 파산할 log-odds
- $\beta_1$: Balance가 1 증가할 때 파산할 log-odds의 증가분
- $\exp(\beta_1)$: Balance가 1 증가할 때 파산의 odds ratio
    - X가 한단위 증가할 때 log-odds가 $\beta_1$ 만큼 증가 = odds가 $e^{\beta_1}$배 변화
- $p(X)$와 X가 선형관계가 아니기 때문에 X의 한단위 증가가 일으키는 p(X)의 증가는 X의 값에 따라 달라짐

#### 회귀계수의 추정 
- Maximum likelihood method의 사용 
$$L(\beta_0, \beta_1) = \prod_{i: y_i=1} p(x_i)\prod_{i': y_{i'}=0} (1-p(x_{i'})) $$

In [2]:
Default.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [3]:
Default = pd.concat([Default,pd.get_dummies(Default.default,prefix="default")],axis=1)

In [4]:
Default.head()

,default,student,balance,income,default_No,default_Yes
0,No,No,729.526495,44361.625074,1,0
1,No,Yes,817.180407,12106.134700,1,0
2,No,No,1073.549164,31767.138947,1,0
3,No,No,529.250605,35704.493935,1,0
4,No,No,785.655883,38463.495879,1,0


In [5]:
model = smf.glm('default_Yes~balance', data=Default, family = sm.families.Binomial()).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:            default_Yes   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -798.23
Date:                Sun, 13 Feb 2022   Deviance:                       1596.5
Time:                        16:52:38   Pearson chi2:                 7.15e+03
No. Iterations:                     9   Pseudo R-squ. (CS):             0.1240
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.6513      0.361    -29.491      0.000     -11.359      -9.943
balance        0.0055      0.000     24.952      0.000       0.005       0.006
==============================================================================
"""

family = sm.families.Binomial()를 사용해야 로지스틱 회귀

- balance의 회귀계수 p-value<0.0001 : 파산확률과 카드잔고 사이에 관계가 있음

In [6]:
np.exp(model.params[1])

1.0055140637295612

- balance가 1 증가하면 파산승산(odds)가 1.005514배 증가한다. 즉, 0.55% 증가한다.
- balance가 100 증가하면 파산승산이 얼만큼 증가하는가? 



답: 100증가하면 파산승산이 1.7330653054483078 만큼 증가

In [7]:
np.exp(model.params[1]*100)
# 1.733배 증가
# 73.3% 증가

1.7330653054483078

In [8]:
# 만약 beta1이 음수라면?
np.exp(-0.05)
# 0.95배 
# 5% 감소

0.951229424500714

In [9]:
model2 = smf.glm('default_Yes~student', data=Default, family = sm.families.Binomial()).fit()
model2.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-3.5041,0.071,-49.554,0.000,-3.643,-3.366
student[T.Yes],0.4049,0.115,3.520,0.000,0.179,0.630


In [10]:
np.exp(model2.params[1])

1.499133209929228

- 학생이면 파산의 odds가 약 50% 증가한다.

## 5.3 Multiple logistic regression
$$ p(X)=\frac{e^{\beta_0 + \beta_1 X_1 + \cdots + \beta_p X_p}}{1+e^{\beta_0 + \beta_1 X_1 + \cdots + \beta_p X_p}}$$
$$\log \left( \frac{p(X)}{1-p(X)}\right) =\beta_0 + \beta_1 X_1 + \cdots + \beta_p X_p$$

In [11]:
model3 = smf.glm('default_Yes~balance + student', data=Default, family = sm.families.Binomial()).fit()
model3.summary().tables[1]

,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-10.7495,0.369,-29.115,0.000,-11.473,-10.026
student[T.Yes],-0.7149,0.148,-4.846,0.000,-1.004,-0.426
balance,0.0057,0.000,24.748,0.000,0.005,0.006


In [12]:
np.exp(model3.params[1])

0.4892519822863241

- 동일한 balance를 가지고 있을 때 학생이면 파산 odds가 약 51% 감소한다.
    - 앞의 결과와 상반된 방향!?
    

#### Confounding effect
- 같은 수준의 balance를 가지고 있을 경우 학생의 파산 odds가 0.5배 
- 학생인 경우 아닌 경우보다 파산 odds가 1.5배
    - 학생일 경우 credit card balance가 더 높은 경향이 있음 
<img src='http://drive.google.com/uc?export=view&id=1N1I1VOhp2dFUcb8FSmUh1njt4F1sf_ri' /><br>